In [1]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
import fitz  # PyMuPDF
from datasets import Dataset
import glob

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text

# Load all PDFs in a folder
pdf_files = glob.glob("Full_Documents/*.pdf")  # Update this path

# Create dataset from extracted texts
data = {"file_name": [], "text": []}

for pdf in pdf_files:
    data["file_name"].append(pdf.split("/")[-1])
    data["text"].append(extract_text_from_pdf(pdf))

dataset = Dataset.from_dict(data)

# Print dataset info
print(dataset)

Dataset({
    features: ['file_name', 'text'],
    num_rows: 13
})


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

langchain_docs = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["file_name"]}) for doc in tqdm(dataset)]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/13 [00:00<?, ?it/s]

In [18]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

In [16]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1" #Depois posso passar o endpoint do Ollama com o modelo aparentemente

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)

In [25]:
import random

N_GENERATIONS = 50 

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = llm_client.text_generation(
        prompt = QA_generation_prompt.format(context=sampled_context.page_content), 
        max_new_tokens=1000)
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

Generating 50 QA couples...


  0%|          | 0/50 [00:00<?, ?it/s]

In [29]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to Gynecologists diagnosing endometriosis, planning a endometriosis therapy or treating a patient with endometriosis.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like QALY, LUNA, AFC or COH and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the diameter range of folicules measured by AFC?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [30]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": llm_client.text_generation(
            prompt= question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
            max_new_tokens=1000
        ),
        "relevance": llm_client.text_generation(
            prompt=question_relevance_critique_prompt.format(question=output["question"]),
            max_new_tokens=1000
        ),
        "standalone": llm_client.text_generation(
            prompt=question_standalone_critique_prompt.format(question=output["question"]),
            max_new_tokens=1000
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue

Generating critique for each QA couple...


  0%|          | 0/46 [00:00<?, ?it/s]

In [35]:
questions = pd.DataFrame(outputs)
questions.to_csv("QA_questions.csv")